In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class PhotoEditor:
    def __init__(self, image_path):
        self.img = cv2.imread(image_path)
        self.history = [self.img.copy()]
        self.log = []

    def _commit(self, action_name):
        self.history.append(self.img.copy())
        self.log.append(action_name)

    def show(self, title='Preview'):
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.title('Original')
        plt.imshow(cv2.cvtColor(self.history[-2], cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(title)
        plt.imshow(cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    def adjust_brightness(self, beta):
        self.img = cv2.convertScaleAbs(self.img, beta=beta)
        self._commit(f"Brightness +{beta}")
        self.show("Brightness Adjusted")

    def adjust_contrast(self, alpha):
        self.img = cv2.convertScaleAbs(self.img, alpha=alpha)
        self._commit(f"Contrast x{alpha}")
        self.show("Contrast Adjusted")

    def to_grayscale(self):
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_GRAY2BGR)
        self._commit("Converted to Grayscale")
        self.show("Grayscale")

    def add_padding(self, top, bottom, left, right, border_type, color=[0, 0, 0]):
        self.img = cv2.copyMakeBorder(self.img, top, bottom, left, right, border_type, value=color)
        self._commit(f"Padded {top},{bottom},{left},{right}")
        self.show("Padded Image")

    def apply_threshold(self, inverse=False):
        gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        thresh_type = cv2.THRESH_BINARY_INV if inverse else cv2.THRESH_BINARY
        _, thresh = cv2.threshold(gray, 127, 255, thresh_type)
        self.img = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
        self._commit("Thresholded " + ("Inverse" if inverse else "Binary"))
        self.show("Thresholded")

    def blend_image(self, second_img_path, alpha):
        second_img = cv2.imread(second_img_path)
        second_img = cv2.resize(second_img, (self.img.shape[1], self.img.shape[0]))
        self.img = ((1 - alpha) * self.img + alpha * second_img).astype(np.uint8)
        self._commit(f"Blended with {second_img_path} α={alpha}")
        self.show("Blended Image")

    def undo(self):
        if len(self.history) > 1:
            self.history.pop()
            self.img = self.history[-1].copy()
            self.log.append("Undo")
            self.show("Undo")
        else:
            print("Nothing to undo.")

    def view_history(self):
        print("\n===== Operation History =====")
        for action in self.log:
            print("•", action)

    def save(self, filename='edited_output.jpg'):
        cv2.imwrite(filename, self.img)
        print(f"✅ Image saved as {filename}")

# ========== Menu Interface ==========

def run_editor():
    path = input("Enter path to image: ")
    editor = PhotoEditor(path)

    while True:
        print("""
===== Mini Photo Editor =====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image
7. Undo Last Operation
8. View History
9. Save and Exit
""")
        choice = input("Choose an option: ")

        try:
            if choice == '1':
                beta = int(input("Enter brightness value (e.g., 50): "))
                editor.adjust_brightness(beta)
                input("\nPress Enter to return to menu...")

            elif choice == '2':
                alpha = float(input("Enter contrast scale (e.g., 1.2): "))
                editor.adjust_contrast(alpha)
                input("\nPress Enter to return to menu...")

            elif choice == '3':
                editor.to_grayscale()
                input("\nPress Enter to return to menu...")

            elif choice == '4':
                print("Border Types:\n0: CONSTANT\n1: REPLICATE\n2: REFLECT")
                bt = int(input("Enter border type (0/1/2): "))
                top = int(input("Top padding: "))
                bottom = int(input("Bottom padding: "))
                left = int(input("Left padding: "))
                right = int(input("Right padding: "))
                btype = [cv2.BORDER_CONSTANT, cv2.BORDER_REPLICATE, cv2.BORDER_REFLECT][bt]
                editor.add_padding(top, bottom, left, right, btype)
                input("\nPress Enter to return to menu...")

            elif choice == '5':
                inv = input("Inverse threshold? (y/n): ").lower() == 'y'
                editor.apply_threshold(inverse=inv)

            elif choice == '6':
                path2 = input("Enter second image path to blend: ")
                alpha = float(input("Alpha (0 to 1): "))
                editor.blend_image(path2, alpha)
                input("\nPress Enter to return to menu...")

            elif choice == '7':
                editor.undo()
                input("\nPress Enter to return to menu...")

            elif choice == '8':
                editor.view_history()
                input("\nPress Enter to return to menu...")

            elif choice == '9':
                fname = input("Enter filename to save: ")
                editor.save(fname)
                break

            else:
                print("Invalid choice.")
                input("\nPress Enter to return to menu...")

        except Exception as e:
            print("⚠️ Error:", e)
            input("\nPress Enter to return to menu...")

# 🔁 Start the CLI
run_editor()

